# Evolutionary Algorithm

## Jupyter Housekeeping

In [1]:
import os
import os.path
import sys
d = os.path.join(os.getcwd(), '..')
print(d)
sys.path.append(d)

C:\Users\Zachary\Documents\GitHub\COMP 3201 - TSP Evolutionary Algorithm\src\..


In [2]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure as Figure
import seaborn as sns
import numpy as np
import pandas as pd

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# https://pandas.pydata.org/pandas-docs/stable/visualization.html
# http://pandas.pydata.org/pandas-docs/version/0.13/visualization.html
# https://matplotlib.org/users/pyplot_tutorial.html

# Set up the style of the graphs, and display a sample graph
graph_style = 5
plt.style.use(plt.style.available[graph_style])  # 5, 14, 22
sns.set_context("paper")

## Imports and Function Declarations

In [3]:
# Standard Imports
# These should not be changed, as they are used to set the 'constant'
# global variables for each module.
from EA_Methods.Pandas_Rep import ParentSelectionMethods as PSM
from EA_Methods.Pandas_Rep import RecombinationMethods as RM
from EA_Methods.Pandas_Rep import MutationMethods as MM
from EA_Methods.Pandas_Rep import SurvivorSelectionMethods as SSM

# Modular Imports
# Modular Problem Definition
# By changing this import, you can easily redifine the problem you are 
# attempting to solve, without having to re-write much of the code.
from Setups.TSP import TSP_PDS as DEF


# Modular Function 'Declarations'
# By swapping out which functions are accessed the entire structure of the 
# EA algorithm can be changed without rewriting code in the main method.
parse_file          =      DEF.read_tsp_file
eval_fitness        =      DEF.euclidean_distance
initialize          =      DEF.random_initialization
parent_selection    =      PSM.roulette
generate_offspring  =      RM.recombination_cut_crossover
apply_mutation      =      MM.permutation_swap
select_survivors    =      SSM.replacement


# Global Variable Initialization
FILENUM             =      1  # Changing this variable changes which file to parse.
genome_length       =      parse_file(FILENUM)
generation_limit    =      10000
population_size     =      60
mating_pool_size    =      population_size//2 if (population_size//2) % 2 == 0 else (population_size//2)+1  # has to be even

tournament_size     =      population_size//10
crossover_rate      =      0.9
crossover_point     =      genome_length//3
mutation_rate       =      0.2


# Key Variable Setters
# These functions typically do not need to be changed, and simply allow
# easy changing of all rates and variables from the variables above.
PSM.set_tournament_size(tournament_size)
RM.set_genome_length(genome_length)
RM.set_crossover_point(crossover_point)
RM.set_crossover_rate(crossover_rate)
MM.set_genome_length(genome_length)
MM.set_mutation_rate(mutation_rate)
MM.set_fitness_function(eval_fitness)
SSM.set_population_size(population_size)
SSM.set_mating_pool_size(mating_pool_size)
DEF.set_fitness_function(eval_fitness)

def set_ops(maximize): # Sets dynamic standard for 'best'
    op = max if maximize else min
    PSM.set_op(op)
    SSM.set_op(op)
    DEF.set_op(op)

DEF.CITIES.head()

,Longitude (Range shifted),Latitude (Range shifted)
City,,
0,-3314.58335,-3633.33335
1,-3247.91665,-3600.00005
2,-2847.91665,449.99995
3,-2547.91665,-683.33335
4,-2547.91665,-1500.00005


## Main Code

In [ ]:
def evolution_algorithm(maximize, print_gens=True, display_freq=None):
    if display_freq is None: display_freq = generation_limit
    
    set_ops(maximize)                                           # Sets dynamic standard for 'best'
    population = initialize(population_size, genome_length)     # Initialize Population
    if print_gens: DEF.start_up_display()                       # Prints City Locations
    
    # Evolution starts
    for generation in range(generation_limit):
        # Generation Info
        if print_gens and (generation % display_freq == display_freq - 1):
            TSP.generation_display(population)                  # Displays Best Path
    
        parents = parent_selection(fitness, mating_pool_size)   # Parent Selection
        offspring = generate_offspring(parents)                 # Parent Recombination
        offspring = apply_mutation(offspring)                   # Mutation Application
        population = select_survivors(population, offspring)    # Survivor Selection
    # Evolution ends
        
    # Final Fitness Info
    op_fit = op(fitness)
    optimal_solutions = [i + 1 for i in range(population_size) if fitness[i] == op_fit]
    print("\nBest solution fitness:", op_fit, "\nNumber of optimal solutions: ", len(optimal_solutions), '/', population_size)
    print("Best solution indexes:\n", optimal_solutions)
    print('\n\n\n\n\n\n')

In [ ]:
freq = min(int(generation_limit * 0.1), 500)
print('\n\nDisplaying information every {} generations.\n\n'.format(freq))
evolution_algorithm(False, True, freq)

## Temporary Method Testing

In [4]:
set_ops(maximize=False)

In [5]:
# random_initialization     - Finished
# heurisitic_initialization - Stub
initialize                  = DEF.random_initialization

population = initialize(population_size, genome_length)
population.head()

,individuals,fitnesses
indexes,,
0,"[7, 16, 23, 4, 9, 8, 15, 18, 1, 26, 17, 11, 28...",116871.074759
1,"[21, 3, 5, 10, 11, 19, 15, 1, 12, 27, 23, 14, ...",99719.080206
2,"[9, 1, 26, 0, 27, 7, 28, 19, 24, 8, 5, 6, 20, ...",112416.307267
3,"[22, 19, 15, 23, 4, 3, 5, 27, 28, 8, 0, 24, 11...",92226.424892
4,"[13, 4, 28, 15, 14, 26, 18, 2, 27, 11, 7, 12, ...",120559.614697


In [6]:
# mps               - In Progress
# tournament        - In Progress
# roulette          - Finished
# random_uniform    - Finished
parent_selection    = PSM.random_uniform

parents = parent_selection(population, mating_pool_size)
parents

,individuals,fitnesses
0,"[2, 13, 11, 12, 1, 18, 5, 8, 20, 4, 15, 26, 17...",115264.224208
1,"[8, 28, 23, 0, 21, 15, 4, 26, 3, 7, 22, 10, 1,...",116884.467655
2,"[20, 18, 14, 15, 7, 12, 13, 22, 25, 0, 4, 21, ...",104084.878880
3,"[28, 4, 17, 2, 19, 22, 11, 3, 12, 25, 9, 26, 1...",113811.508418
4,"[5, 11, 8, 19, 17, 21, 13, 16, 6, 20, 14, 25, ...",129622.195805
5,"[12, 5, 27, 20, 18, 6, 16, 13, 9, 10, 23, 25, ...",101789.482356
6,"[3, 0, 8, 2, 13, 18, 16, 27, 4, 6, 10, 25, 21,...",84240.472272
7,"[5, 0, 3, 7, 4, 24, 8, 9, 28, 6, 13, 21, 19, 1...",107080.131319
8,"[16, 14, 1, 11, 17, 9, 8, 5, 23, 22, 10, 15, 7...",113795.612011
9,"[13, 22, 17, 2, 24, 18, 26, 6, 8, 7, 21, 5, 23...",124142.371284


In [7]:
# recombination_cut_crossover   - In Progress  # This fails to return a dataframe
# recombination_pmx_crossover   - Stub
# recombination_edge_crossover  - Stub
# recombination_order_crossover - Stub
generate_offspring              = RM.recombination_cut_crossover

offspring = generate_offspring(parents)
offspring

0     ([2, 13, 11, 12, 1, 18, 5, 8, 20, 7, 22, 10, 1...
1     ([8, 28, 23, 0, 21, 15, 4, 26, 3, 17, 9, 16, 7...
2     ([20, 18, 14, 15, 7, 12, 13, 22, 25, 0, 4, 21,...
3     ([28, 4, 17, 2, 19, 22, 11, 3, 12, 0, 21, 23, ...
4     ([5, 11, 8, 19, 17, 21, 13, 16, 6, 10, 23, 25,...
5     ([12, 5, 27, 20, 18, 6, 16, 13, 9, 14, 25, 4, ...
6     ([3, 0, 8, 2, 13, 18, 16, 27, 4, 6, 21, 19, 14...
7     ([5, 0, 3, 7, 4, 24, 8, 9, 28, 6, 10, 25, 21, ...
8     ([16, 14, 1, 11, 17, 9, 8, 5, 23, 7, 21, 28, 1...
9     ([13, 22, 17, 2, 24, 18, 26, 6, 8, 10, 15, 7, ...
10    ([11, 6, 15, 19, 13, 22, 8, 23, 21, 20, 2, 0, ...
11    ([3, 25, 27, 21, 22, 18, 12, 17, 1, 5, 9, 2, 4...
12    ([5, 16, 11, 18, 25, 20, 22, 13, 27, 9, 4, 0, ...
13    ([19, 13, 11, 8, 15, 26, 7, 10, 20, 17, 1, 6, ...
14    ([21, 3, 5, 10, 11, 19, 15, 1, 12, 17, 26, 6, ...
15    ([5, 16, 11, 18, 25, 20, 22, 13, 27, 23, 14, 1...
16    ([19, 13, 11, 8, 15, 26, 7, 10, 20, 9, 4, 18, ...
17    ([17, 11, 3, 15, 18, 16, 20, 4, 0, 9, 1, 2

In [9]:
# permutation_swap       - Finished  # There may be some error here.
# permutation_insert     - Stub
# permutation_inversion  - Stub
# permutation_scramble   - Stub
apply_mutation           = MM.permutation_swap

offspring = apply_mutation(offspring)
offspring

TypeError: method_randomizer() got an unexpected keyword argument 'axis'

In [10]:
# mu_plus_lambda    - Finished
# replacement       - Finished
# random_uniform    - Finished
select_survivors    = SSM.mu_plus_lambda

population = select_survivors(population, offspring)
population

TypeError: Can only append a Series if ignore_index=True or if the Series has a name

## Helper Functions

In [ ]:
# Sahara
parse_file(1)
DEF.start_up_display()
#TSP.brute_force_solver()

In [ ]:
# Uraguay
parse_file(2)
DEF.start_up_display()
#TSP.brute_force_solver()

In [ ]:
# Canada
parse_file(3)
DEF.start_up_display()
#TSP.brute_force_solver()